In [ ]:
""" LSTM """
input_size = X_train.shape[-1]
num_layers = 2
hidden_size = 64

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # (lstm): LSTM(1, 64, num_layers=2, batch_first=True)

        self.linear = nn.Linear(hidden_size, 1)
        # (linear): Linear(in_features=64, out_features=1, bias=True)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.linear(out)

model = LSTMModel(input_size, hidden_size, num_layers).to(device)

In [ ]:
"""
    # llm의 모델 결과와 약간 비슷한데 다르다.
    차이점 잘 구분할 것.
"""

def train(model, train_loader, test_loader):
    train_hist = []
    test_hist = []
    num_epochs = 10
    loss_fn = nn.MSELoss(reduction='mean')
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_test_loss = 0.0

        # train
        model.train()
        for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        pred = model(batch_x)[:, -1, 0]
        # llm은 [:, -1, :] 인데 ts는 [:, -1, 0]이다. 헷갈리지 말자.
        loss = loss_fn(pred, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    avg_loss = total_train_loss / len(train_loader)
    train_hist.append(avg_loss)

    # evaluate
    model.eval()
    with torch.no_grad():
        for test_x, test_y in test_loader:
            test_x, test_y = test_x.to(device), test_y.to(device)
            test_pred = model(test_x)[:, -1, 0]
            test_loss = loss_fn(test_pred, test_y)
            total_test_loss += test_loss.item()

    avg_test_loss = total_test_loss / len(test_loader)
    test_hist.append(avg_test_loss)

   

In [ ]:
def plot_forecasting(model, X_test, y_test):
  model.eval()
  num_forecast_steps = 30
  input_data = X_test[-num_forecast_steps].cpu().numpy().squeeze()

  forecasted_values = []
  with torch.no_grad():
    for i in range(2 * num_forecast_steps):
      input_tensor = torch.as_tensor(input_data).view(1, -1, 1).to(device)
      predicted = model(input_tensor)[0, -1, 0].item()

      forecasted_values.append(predicted)
      input_data = np.roll(input_data, shift=-1)

      if i < num_forecast_steps:
        input_data[-1] = y_test[-num_forecast_steps + i]
      else:
        input_data[-1] = predicted

In [ ]:
class Conv1DModel(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(Conv1DModel, self).__init__()
    self.conv1d = nn.Conv1d(in_channels=input_size, \
                            out_channels=hidden_size, \
                            kernel_size=2, stride=1)
    self.fc = nn.Linear(hidden_size, 1)

  def forward(self, x):
    x = x.transpose(1, 2)
        # Conv1d는 (batch, channels, length) 형태를 기대한다.
        # (batch, seq_len, input_size) → (batch, input_size, seq_len)로 바꿔줌.
    x = self.conv1d(x) # 결과 shape: (batch, hidden_size, seq_len-1)
    x = x.transpose(1, 2)
        # 다시 (batch, seq_len-1, hidden_size)로 바꿔서 Linear에 넣기 좋게 만든다.
    return self.fc(x)

In [ ]:
"""
    RNN도 한번 봐두지만 input_size가 X_train.shape[-1]이 되는게 다른 분야와 차이점인듯
"""
class RNNModel(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
      super(RNNModel, self).__init__()
      self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
      self.fc = nn.Linear(hidden_size, 1)

  def forward(self, x):
      out, _ = self.rnn(x)
      return self.fc(out)

input_size = X_train.shape[-1]
num_layers = 2
hidden_size = 64

In [ ]:
def create_enc_dec_sequences(data):
    features, labels = [], []
    for i in range(len(data) - sequence_length - target_len):
        features.append(data[i:i + sequence_length])
        labels.append(data[i + sequence_length : i + sequence_length + target_len])
        # feature 다음것부터 target_len개만큼 가져온다.

    features = np.array(features, dtype=np.float32)
    labels = np.array(labels, dtype=np.float32)
    # noarray로 한번 바꾸고 
    # 꼭 2단계로 할 필요는 없는데 타입을 명확히 float32로 맞추고, shape을 안정적으로 정리하기 위해

    features = torch.tensor(features, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32)
    #tensor로 바꿔준다.
    return features, labels

X_train_, y_train_ = create_enc_dec_sequences(train_scaled)
X_test_, y_test_ = create_enc_dec_sequences(test_scaled)

train_loader_ = to_loader(X_train_, y_train_, batch_size, shuffle=True)
test_loader_ = to_loader(X_test_, y_test_, batch_size, shuffle=False)

In [ ]:
class EncoderRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(EncoderRNN, self).__init__()
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)

  def forward(self, x):
    _, h = self.rnn(x)
    return h

In [ ]:
class DecoderRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(DecoderRNN, self).__init__()
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, input_size)

  def forward(self, x, h):
    out, h = self.rnn(x, h)
    out = self.fc(out)
    return out, h

In [ ]:
class RNNRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(RNNRNN, self).__init__()
    self.encoder = EncoderRNN(input_size, hidden_size, num_layers)
    self.decoder = DecoderRNN(input_size, hidden_size, num_layers)

  def forward(self, source, target_len): # source shape:(batch, seq_len, input_size)
    h = self.encoder(source) # h shape: (num_layers, batch, hidden_size)
    predictions = []
    input = source[:, -1, :].unsqueeze(1) # shape: (batch, input_size) > (batch, 1, input_size)
    for t in range(target_len):
        out, h = self.decoder(input, h) # out: (batch, 1, input_size) / h: (num_layers, batch, hidden_size)
        predictions.append(out.squeeze(1)) # out : (batch, 1, input_size) → (batch, input_size)
        input = out # (batch, input_size)
    outputs = torch.stack(predictions, dim=1) # (batch, target_len, input_size)

    return outputs


In [ ]:
def train_(model, train_loader, test_loader):
    train_hist = []
    test_hist = []
    num_epochs = 10
    loss_fn = nn.MSELoss(reduction='mean')
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_test_loss = 0.0

        # train
        model.train()
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            pred = model(batch_x, batch_y.shape[1]) # batch_y.shape[1] = target_len
            loss = loss_fn(pred, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_loss = total_train_loss / len(train_loader)
        train_hist.append(avg_loss)

        # evaluate
        model.eval()
        with torch.no_grad():
            for test_x, test_y in test_loader:
            test_x, test_y = test_x.to(device), test_y.to(device)
            test_pred = model(test_x, target_len) # tesy_y.shape[1]로 해야 하는거 아닌지?
            test_loss = loss_fn(test_pred, test_y)
            total_test_loss += test_loss.item()

        avg_test_loss = total_test_loss / len(test_loader)
        test_hist.append(avg_test_loss)

        print(f'Epoch {epoch + 1:2d}/{num_epochs} - Training Loss: {avg_loss:.4f}, Test Loss: {avg_test_loss:.4f}')

train_(model_, train_loader_, test_loader_)

In [ ]:
def plot_forecasting(model, X_test, y_test):
    model.eval()
    num_forecast_steps = 30

    with torch.no_grad():
        for i in range(1, min(num_forecast_steps, len(X_test)) + 1):
            input_tensor = X_test[-i].unsqueeze(0).to(device)
            predicted = model(input_tensor, target_len) # (batch, target_len, input_size)
            forecasted_values = predicted.squeeze(0).cpu().numpy() # : (target_len, input_size)
            forecasted_values = scaler.inverse_transform( # 학습 전에 MinMaxScaler나 StandardScaler로 정규화했던 값을 원래 스케일로 
                forecasted_values.reshape(-1, 1) # (target_len, input_size) → (target_len * input_size, 1)
            ).flatten()
            idx = test_data.index[-i - target_len + 1 : -i + 1 if i > 1 else None]
            plt.plot(idx, forecasted_values, color="red", alpha=0.6,
                     label="forecasted values" if i == 1 else "")

    plt.plot(test_data.index[-100:-39], test_data[-100:-39],
             label="test_data", color="b")
    plt.plot(test_data.index[-39:], test_data.iloc[-39:],
             label="actual values", color="green")

    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.legend()
    plt.title('Time Series Forecasting')
    plt.grid(True)
    plt.show()


plot_forecasting(model_, X_test_, y_test_)

In [ ]:
def test_(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        X_test = X_test.to(device)
        y_hat = model(X_test, target_len)
        test_predictions = y_hat[:, :, 0]

    test_predictions = test_predictions.cpu().numpy()
    y_test = y_test.cpu().numpy().reshape(-1, target_len)

    rmse = root_mean_squared_error(y_test, test_predictions)
    mape = mean_absolute_percentage_error(y_test, test_predictions)

    return rmse, mape

rmse, mape = test_(model_, X_test_, y_test_)